# Extract Audio Feautures


In [ ]:
import sys
import pickle
import pathlib
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

import audiofile
import opensmile

from util import *


In [2]:
print("Version: ", tf.__version__)
# tf.compat.v1.enable_eager_execution() # Enable eager execution
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.experimental.list_physical_devices('GPU') else "NOT AVAILABLE")
# print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

## File paths

In [3]:
HOME_DIR = pathlib.Path.cwd()

# feature_path = HOME_DIR / 'data' / 'processed'/ 'IEMOCAP' / 'extracted_feature.pk'
feature_path = 'D:/extracted_features.pk'

label_path = HOME_DIR / 'data' / 'processed' / 'IEMOCAP' / 'FC_label.txt'
processed_id_path = HOME_DIR / 'data' / 'processed' / 'IEMOCAP' / 'processed_ids.txt'
dataset_path = HOME_DIR / 'data' / 'raw'


In [3]:
def configure_openSmile(**kwargs):
    '''
    Configure OpenSmile.

        Parameters:
            folder_list (list): A list of folder paths to extract features

        Returns:
            smile (obj): Returns configured smile object
    '''
    
    smile = opensmile.Smile(
        feature_set=opensmile.FeatureSet.ComParE_2016,
        feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
        loglevel=4,
        logfile='smile.log',
        verbose=True,

    )
    return smile


In [4]:
def extract_feautre(folder_list):
    '''
    Extracts audio feature for each file in file path.

        Parameters:
            folder_list (list): A list of filepaths to extract audio features

        Returns:
            features (Dataframe): Extracted features from USC-Iemocap database
    '''
    
    smile = configure_openSmile()
    try:
        features = smile.process_files(folder_list)
        return features

    except FileNotFoundError as e:
        print(f'Error --> {e}')


In [5]:
def z_score(dataframe, inplace=False):
    '''
    Apply z-score (standardization) to the columns of the database.

        Parameters:
            data: Data to apply z-score
            inplace (bool): Modify the dataframe inplace. Default (False)

        Returns:
            dataframe: Z-scores applied on each columns of dataframe
    '''

    if not inplace:
        dataframe = dataframe.copy()

    # apply the z-score method
    for column in dataframe.columns:
        dataframe[column] = (dataframe[column] - dataframe[column].mean()
                             ) / dataframe[column].std()

    print('Finished standardizing (z-score) data')
    return dataframe


In [6]:
def pad_signal_data(data, full_path, maxlen=3409):
    '''
    Pad each signal with zeros for eqaul data dimenstions.

        Parameters:
            data (Pandas DataFrame): Signal list to be padded
            maxlen = Maximum length of sequence after pappding

        Returns:
            padded_dataframe (DataFrame): Padded dataframe
    '''

    padded_dataframe, frames = pd.DataFrame(columns=data.columns), []

    for file in full_path:
        x_ = data.loc[file]
        x_.reset_index(inplace=True, drop=True)  # Remove multi-index
        rows, cols = x_.shape

        if rows < maxlen:
            # rows = maxlen - rows

            zeros = np.zeros((maxlen, cols), dtype=float)
            zeros_df = pd.DataFrame(data=zeros, columns=data.columns)
            x_ = x_.combine_first(zeros_df)

        frames.append(x_)

    padded_dataframe = pd.concat(frames, keys=range(len(full_path)))

    print('Finished padding data')
    return padded_dataframe


In [8]:
def shuffle_dataset(features, labels):
    # random.seed(24)

    shuffled_data = list(zip(features, labels))
    random.shuffle(shuffled_data)
    shuf_features, shuf_label = zip(*shuffled_data)
    return shuf_features, shuf_label

## Extract and save features

In [12]:
list_files = []

for x in range(1, 6):
    sess_title = 'Session' + str(x)

    path = f'./data/raw/IEMOCAP_full_release/{sess_title}/sentences/wav'
#     path = dataset_path / 'IEMOCAP_full_release' / f'{sess_title}' / 'sentences' / 'wav'

    file_search(str(path), list_files)
    list_files = sorted(list_files)

    print(f"{sess_title}, # Num of files: {len(list_files)}")


In [ ]:
# Extract audio features
features = extract_feautre(list_files)

# Save features
with open('D:/extracted_features.pk', 'wb') as feat:
    pickle.dump(features, feat)


## Load and prepare features

In [10]:
label = []

with open(str(feature_path), 'rb') as pickle_file:
    with open(str(label_path)) as f2:
        data = pickle.load(pickle_file)
        category = f2.readlines()

label = [y.strip() for y in category]

## Dataset info

In [11]:
full_ids = []
with open(str(processed_id_path)) as f:
    full_ids = f.readlines()
full_ids = [x.strip() for x in full_ids]

## Get list of filenames in category, (Angry, Happy, Sad, Neutral)

In [13]:
full_path = []

for file_ in full_ids:
    for file2 in list_files:
        if file_+'.wav' == file2.split('\\')[-1]:
            full_path.append(file2)
            break

In [14]:
data.head()

F0final_sma  \
file                                               start           end                            
./data/raw/IEMOCAP_full_release/Session1/senten... 00:00:00        00:00:00.060000          0.0   
                                                   00:00:00.010000 00:00:00.070000          0.0   
                                                   00:00:00.020000 00:00:00.080000          0.0   
                                                   00:00:00.030000 00:00:00.090000          0.0   
                                                   00:00:00.040000 00:00:00.100000          0.0   

                                                                                    voicingFinalUnclipped_sma  \
file                                               start           end                                          
./data/raw/IEMOCAP_full_release/Session1/senten... 00:00:00        00:00:00.060000                   0.617895   
                                                   00:00:00.010000 00:00:00.070000                   0.621465   
                                                   00:00:00.020000 00:00:00.080000                   0.625829   
                                                   00:00:00.030000 00:00:00.090000                   0.625295   
                                                   00:00:00.040000 00:00:00.100000                   0.603681   

                                                                                    jitterLocal_sma  \
file                                               start           end                                
./data/raw/IEMOCAP_full_release/Session1/senten... 00:00:00        00:00:00.060000              0.0   
                                                   00:00:00.010000 00:00:00.070000              0.0   
                                                   00:00:00.020000 00:00:00.080000              0.0   
                                                   00:00:00.030000 00:00:00.090000              0.0   
                                                   00:00:00.040000 00:00:00.100000              0.0   

                                                                                    jitterDDP_sma  \
file                                               start           end                              
./data/raw/IEMOCAP_full_release/Session1/senten... 00:00:00        00:00:00.060000            0.0   
                                                   00:00:00.010000 00:00:00.070000            0.0   
                                                   00:00:00.020000 00:00:00.080000            0.0   
                                                   00:00:00.030000 00:00:00.090000            0.0   
                                                   00:00:00.040000 00:00:00.100000            0.0   

                                                                                    shimmerLocal_sma  \
file                                               start           end                                 
./data/raw/IEMOCAP_full_release/Session1/senten... 00:00:00        00:00:00.060000               0.0   
                                                   00:00:00.010000 00:00:00.070000               0.0   
                                                   00:00:00.020000 00:00:00.080000               0.0   
                                                   00:00:00.030000 00:00:00.090000               0.0   
                                                   00:00:00.040000 00:00:00.100000               0.0   

                                                                                    logHNR_sma  \
file                                               start           end                           
./data/raw/IEMOCAP_full_release/Session1/senten... 00:00:00        00:00:00.060000      -100.0   
                                                   00:00:00.010000 00:00:00.070000      -100.0   
                                               

# Pre-process extracted signal sequence

In [15]:
# Pad data
padded_data = pad_signal_data(data, full_path)


In [16]:
# Z-norm data
data_norm = z_score(padded_data, inplace=True)

In [17]:
arr = data_norm.values.reshape(5531, 3409, 65)


In [19]:
# Save pre-processed data
np.savez_compressed('D:/processed_features.npz', arr)
